In [1]:
import pandas as pd 
import numpy as np
import os 
import seaborn as sns 
import matplotlib.pyplot as plt 
import scipy.stats as stats 

## Set input and output folders 

In [2]:
# analysis folder version  
analysis_version = '006'


In [3]:
out_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis',
                        analysis_version, 
                        '002_video_vs_mat_metrics')

if not os.path.exists(out_path): 
    os.makedirs(out_path)

### Load Clean Data - no missing BW Data 
May be missing video data - see excel with counts 

Only participants with MS 

In [4]:
# PWS 
zv_pws_bw_clean_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis', 
                                    analysis_version, 
                                    '000_merged_cleaned_data\zv_bw_merged_gait_vertical_PWS_1_clean.csv')
zv_pws_bw_clean_df = pd.read_csv(zv_pws_bw_clean_path, index_col = 0)

# FW 
zv_fw_bw_clean_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis', 
                                    analysis_version, 
                                   '000_merged_cleaned_data\zv_bw_merged_gait_vertical_FW_1_clean.csv') 
zv_fw_bw_clean_df = pd.read_csv(zv_fw_bw_clean_path, index_col = 0) 

# Home Videos 
hv_bw_clean_path = os.path.join(r'C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis', 
                                analysis_version, 
                                '000_merged_cleaned_data\hv_bw_merged_clean.csv') 

hv_bw_clean_df = pd.read_csv(hv_bw_clean_path, index_col = 0) 

## Columns to compare 
Column pairs to evaluate metrics that should/count be 1:1, not proxy velocity measures  
For each of the column pairs below (zv 1 vs bw1, zv 2 vs bw 3, etc), run and save correlation 

In [5]:
# zeno video metrics 
zv_colnames = ['stride_time_median_sec_pose_zv', 
               'stride_time_mean_sec_pose_zv',
               'gait_cycle_time_sec_median_pose_zv',
               'stride_time_cv_pose_zv', 
               'mean_cadence_step_per_min_pose_zv',
               'tot_dsupport_per_mean_pose_zv',
               'tot_dsupport_per_median_pose_zv',
               'singlesupport_per_mean_pose_zv',
               'singlesupport_per_median_pose_zv',
               'stride_width_median_cm_pose_zv',
               'stride_width_mean_cm_pose_zv',
               'stride_width_std_pose_zv']

# home video metrics 
hv_colnames = ['stride_time_median_sec_pose_hv', 
               'stride_time_mean_sec_pose_hv',
               'gait_cycle_time_sec_median_pose_hv', 
               'stride_time_cv_pose_hv', 
               'mean_cadence_step_per_min_pose_hv',
               'tot_dsupport_per_mean_pose_hv',
               'tot_dsupport_per_median_pose_hv',
               'singlesupport_per_mean_pose_hv',
               'singlesupport_per_median_pose_hv',
               'stride_width_median_cm_pose_hv',
               'stride_width_mean_cm_pose_hv',
               'stride_width_std_pose_hv']

# Zeno mat preferred walking speed metrics 
bw_pws_colnames = ['PWS_stridetimesecmean', 
                   'PWS_stridetimesecmean',
                   'PWS_stridetimesecmean',
                   'PWS_stridetimeseccv',
                   'PWS_cadencestepsminmean',
                   'PWS_totaldsupportmean',
                   'PWS_totaldsupportmean',
                   'PWS_singlesupportmean', 
                   'PWS_singlesupportmean',
                   'PWS_stridewidthcmmean',
                   'PWS_stridewidthcmmean',
                   'PWS_stridewidthcmsd']

# Zeno mat fast walking speed metrics 
bw_fw_colnames = ['FW_stridetimesecmean', 
                  'FW_stridetimesecmean', 
                  'FW_stridetimesecmean', 
                  'FW_stridetimeseccv',
                   'FW_cadencestepsminmean',
                   'FW_totaldsupportmean',
                  'FW_totaldsupportmean',
                   'FW_singlesupportmean', 
                  'FW_singlesupportmean',
                   'FW_stridewidthcmmean',
                  'FW_stridewidthcmmean',
                   'FW_stridewidthcmsd']

units = ['seconds',
         'seconds',
         'seconds',
         'CV%',
         'steps/min',
         '%',
         '%',
         '%', 
         '%',
         'cm',
         'cm',
         'cm']

### Correlation - compare metrics from two data sources 

In [6]:
# function - correlation 
def metric_correlation(df, video_columns, bw_columns, output_folder_path, subfolder_name): 
    if not os.path.exists(os.path.join(output_folder_path, subfolder_name)):
        os.makedirs(os.path.join(output_folder_path, subfolder_name))
    
    # create empty list to store results 
    corr_results_all = [] 
    clean_df = pd.DataFrame() 
    
    for metric_i, current_metric in enumerate(video_columns): 
        current_vid_col = video_columns[metric_i]
        current_bw_col = bw_columns[metric_i]

        # Drop rows with NaN values in either column - required to run spearman r 
        clean_df = df.dropna(subset=[current_vid_col, current_bw_col])
        
        # plot 
        sns.scatterplot(x = current_bw_col, y = current_vid_col, 
                        data = clean_df, 
                        alpha = 0.75,
                        hue = 'demographic_diagnosis', 
                        palette = {'MS' : 'royalblue', 
                                   'HC' : 'orange'})
        
        # Set the x and y axis limits to the same range 
        min_val = min(clean_df[current_vid_col].min(), clean_df[current_vid_col].min())
        max_val = max(clean_df[current_vid_col].max(), clean_df[current_vid_col].max())
        plt.xlim(min_val - (min_val * .1), max_val + (max_val * .1))
        plt.ylim(min_val - (min_val * .1), max_val + (max_val * .1))
        # straight line of perfect agreement 
        plt.plot([min_val, max_val], [min_val, max_val], color='lightgrey')
        plt.title(subfolder_name)
        plt.legend(loc = 'upper right')
        plt.savefig(os.path.join(output_folder_path, 
                                 subfolder_name, 
                                 str(current_vid_col + '_vs_' + current_bw_col + '.png')))
        plt.close()

        # run spearman correlation and append   
        statistic, p_value = stats.spearmanr(clean_df[current_bw_col], clean_df[current_vid_col])
        corr_results_all.append({'bw_column': current_bw_col, 
                                 'video_column': current_vid_col, 
                                 'corr_method': 'spearman' , 
                                 'rs': statistic, 
                                 'p_value' : p_value,
                                 'n_pairs': len(clean_df)})

    # Create DataFrame with results
    corr_results_df = pd.DataFrame(corr_results_all)
    corr_results_df = corr_results_df.round(2)

    return corr_results_df


In [7]:
# set correlation output folder 
corr_out_path = os.path.join(out_path, 'correlation')
print(corr_out_path)

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis\006\002_video_vs_mat_metrics\correlation


In [8]:
# PWS 
zv_pws_corr_results_df = metric_correlation(df = zv_pws_bw_clean_df, 
                                     video_columns = zv_colnames, 
                                     bw_columns = bw_pws_colnames, 
                                     output_folder_path = corr_out_path, 
                                     subfolder_name = 'zeno_pws_scatterplots')
zv_pws_corr_results_df.to_csv(os.path.join(corr_out_path, 'zeno_pws_spearman_corr.csv'))

In [9]:
# FW
zv_fw_corr_results_df = metric_correlation(df = zv_fw_bw_clean_df, 
                                     video_columns = zv_colnames, 
                                     bw_columns = bw_fw_colnames, 
                                     output_folder_path = corr_out_path, 
                                     subfolder_name = 'zeno_fw_scatterplots')
zv_fw_corr_results_df.to_csv(os.path.join(corr_out_path, 'zeno_fw_spearman_corr.csv'))

In [10]:
# Home Videos 
hv_corr_results_df = metric_correlation(df = hv_bw_clean_df, 
                                    video_columns = hv_colnames, 
                                     bw_columns = bw_pws_colnames, 
                                     output_folder_path = corr_out_path, 
                                     subfolder_name = 'home_scatterplots')
hv_corr_results_df.to_csv(os.path.join(corr_out_path, 'home_spearman_corr.csv'))

### Mean Absolute Error - compare metrics from two data sources 

In [11]:
def calculate_metric_mean_error(df, video_columns, bw_columns, units, output_folder_path, subfolder_name):
    
    if not os.path.exists(os.path.join(output_folder_path, subfolder_name)):
        os.makedirs(os.path.join(output_folder_path, subfolder_name)) 
        
    mean_error_all = [] 

    for metric_i, current_metric in enumerate(video_columns): 
        current_vid_col = video_columns[metric_i]
        current_bw_col = bw_columns[metric_i]
        current_unit = units[metric_i]

        # Drop rows with NaN values in either column 
        clean_df = df.dropna(subset=[current_vid_col, current_bw_col])

        current_metric_diff = clean_df[current_bw_col] - clean_df[current_vid_col]
        current_mean_diff = current_metric_diff.mean()
        current_abs_mean_diff = abs(current_metric_diff).mean()

        # calculate mean ground truth data 
        bw_mean = clean_df[current_bw_col].mean()
        mean_err_per = (current_mean_diff / bw_mean) * 100 
        mae_per = (current_abs_mean_diff / bw_mean) * 100 

        # plot 
        fig, ax1 = plt.subplots()
        sns.boxplot(y=current_metric_diff, ax=ax1, fill = False, dodge = True, fliersize = 0)
        sns.stripplot(y = current_metric_diff, ax = ax1, color = 'black', dodge = True)
        fig.suptitle(subfolder_name)
        ax1.set_title(current_bw_col + ' - ' + current_vid_col)
        # center plot at zero
        ymin, ymax = plt.ylim()
        plt.ylim(min(ymin, -ymax), max(ymax, -ymin))
        plt.ylabel(current_unit)
        # add line at zero
        plt.axhline(y=0, color='grey', linestyle='--')
        plt.tight_layout()
        plt.savefig(os.path.join(output_folder_path, 
                                 subfolder_name,
                                 str(current_vid_col + '_vs_' + current_bw_col + '_diff_box.png')))
        plt.close()

        # mean difference 
        mean_error_all.append({'bw_column': current_bw_col, 
                               'video_column': current_vid_col,
                               'n_pairs' : len(clean_df), 
                               'bw_metric_mean' : bw_mean,
                               'mean_error': current_mean_diff, 
                               'mean_abs_error' : current_abs_mean_diff, 
                               'mean_error_%_of_mean' : mean_err_per,
                               'mae_%_of_mean' : mae_per})

    
     # Create DataFrame with results
    mean_error_df = pd.DataFrame(mean_error_all)
    mean_error_df = mean_error_df.round(2)
    
    return mean_error_df

In [12]:
# set mean error output folder 
mae_out_path = os.path.join(out_path, 'mean_error')
print(mae_out_path)

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis\006\002_video_vs_mat_metrics\mean_error


In [13]:
# PWS 
zv_pws_mae_results_df = calculate_metric_mean_error(df = zv_pws_bw_clean_df, 
                                                    video_columns = zv_colnames, 
                                                    bw_columns = bw_pws_colnames, 
                                                    units = units, 
                                                    output_folder_path = mae_out_path, 
                                                    subfolder_name = 'zeno_pws_boxplots')
zv_pws_mae_results_df.to_csv(os.path.join(mae_out_path, 'zeno_pws_errors.csv'))

In [14]:
# FW 
zv_fw_mae_results_df = calculate_metric_mean_error(df = zv_fw_bw_clean_df, 
                                                    video_columns = zv_colnames, 
                                                    bw_columns = bw_fw_colnames, 
                                                    units = units, 
                                                    output_folder_path = mae_out_path, 
                                                    subfolder_name = 'zeno_fw_boxplots')
zv_fw_mae_results_df.to_csv(os.path.join(mae_out_path, 'zeno_fw_errors.csv'))

In [15]:
# Home Videos  
hv_mae_results_df = calculate_metric_mean_error(df = hv_bw_clean_df, 
                                                    video_columns = hv_colnames, 
                                                    bw_columns = bw_pws_colnames, 
                                                    units = units, 
                                                    output_folder_path = mae_out_path, 
                                                    subfolder_name = 'home_boxplots')
hv_mae_results_df.to_csv(os.path.join(mae_out_path, 'home_errors.csv'))

### Bland Altman Plots 

In [16]:
def bland_altman_plot(df, video_columns, bw_columns, col_color_key, units, output_folder_path, subfolder_name):
    
    if not os.path.exists(os.path.join(output_folder_path, subfolder_name)):
        os.makedirs(os.path.join(output_folder_path, subfolder_name)) 

    for metric_i, current_metric in enumerate(video_columns): 
        current_bw_col = bw_columns[metric_i]
        current_vid_col = video_columns[metric_i]
        current_unit = current_unit = units[metric_i]

        clean_df = df.dropna(subset=[current_vid_col, current_bw_col])
    
        # Compute the mean and the difference
        mean_measurements = (clean_df[current_bw_col] + clean_df[current_vid_col]) / 2
        diff_measurements = clean_df[current_bw_col] - clean_df[current_vid_col]  # Difference between measurements

        # Mean difference and standard deviation of the difference
        mean_diff = np.mean(diff_measurements)
        std_diff = np.std(diff_measurements)

        # Plot the data
        plt.figure(figsize=(10, 6))
        plt.scatter(mean_measurements, diff_measurements, alpha=1, c = clean_df[col_color_key])
        plt.colorbar(location = 'left', label = col_color_key)
    
        # Add mean difference line and limits of agreement (±1.96*std)
        plt.axhline(mean_diff, color='black', linestyle='--', label=f'Mean diff: {mean_diff:.2f}')
        plt.axhline(mean_diff + 1.96 * std_diff, color='red', linestyle='--', label=f'+1.96 SD: {mean_diff + 1.96 * std_diff:.2f}')
        plt.axhline(mean_diff - 1.96 * std_diff, color='blue', linestyle='--', label=f'-1.96 SD: {mean_diff - 1.96 * std_diff:.2f}')
        plt.axhline(y=0, color='grey', linestyle='--')
    
        # Labels and title
        plt.xlabel('Mean of Zeno Mat vs Video Pose Metric (' + current_unit + ')') 
        plt.ylabel('Zeno mat - Video Pose Metric (' + current_unit + ')')
        plt.title(current_bw_col + ' vs ' + current_vid_col)
        plt.legend(scatterpoints=1, frameon=True, labelspacing=1, title='T25FW Avg', 
                   loc='center left', bbox_to_anchor = (1, 0.5))
        plt.savefig(os.path.join(output_folder_path, 
                                 subfolder_name,
                                 str(current_vid_col + '_vs_' + current_bw_col + '_blandalt.png')),
                    bbox_inches='tight')
        plt.close()

In [17]:
# set bland altman output folder 
bland_alt_out_path = os.path.join(out_path, 'bland_altman')
print(bland_alt_out_path)

C:\Users\mmccu\Box\MM_Personal\5_Projects\BoveLab\3_Data_and_Code\gait_bw_zeno_home_analysis\006\002_video_vs_mat_metrics\bland_altman


In [18]:
# PWS 
# color by EDSS 
bland_altman_plot(df = zv_pws_bw_clean_df, 
                  video_columns = zv_colnames, 
                  bw_columns = bw_pws_colnames, 
                  col_color_key = 'clean_EDSS', 
                  units = units, 
                  output_folder_path = bland_alt_out_path, 
                  subfolder_name = 'zeno_pws_bland_alt_by_edss')

# color by T25FW 
bland_altman_plot(df = zv_pws_bw_clean_df, 
                  video_columns = zv_colnames, 
                  bw_columns = bw_pws_colnames, 
                  col_color_key = 'clean_T25FW_Avg', 
                  units = units, 
                  output_folder_path = bland_alt_out_path, 
                  subfolder_name = 'zeno_pws_bland_alt_by_t25fw')

# color by that videos velocity from Zeno mat  
bland_altman_plot(df = zv_pws_bw_clean_df, 
                  video_columns = zv_colnames, 
                  bw_columns = bw_pws_colnames, 
                  col_color_key = 'PWS_velocitycmsecmean', 
                  units = units, 
                  output_folder_path = bland_alt_out_path, 
                  subfolder_name = 'zeno_pws_bland_alt_by_pws_vel')

In [19]:
# FW 
# color by EDSS 
bland_altman_plot(df = zv_fw_bw_clean_df, 
                  video_columns = zv_colnames, 
                  bw_columns = bw_fw_colnames, 
                  col_color_key = 'clean_EDSS', 
                  units = units, 
                  output_folder_path = bland_alt_out_path, 
                  subfolder_name = 'zeno_fw_bland_alt_by_edss')

# color by T25FW 
bland_altman_plot(df = zv_fw_bw_clean_df, 
                  video_columns = zv_colnames, 
                  bw_columns = bw_fw_colnames, 
                  col_color_key = 'clean_T25FW_Avg', 
                  units = units, 
                  output_folder_path = bland_alt_out_path, 
                  subfolder_name = 'zeno_fw_bland_alt_by_t25fw')

# color by fast walking video velocity
bland_altman_plot(df = zv_fw_bw_clean_df, 
                  video_columns = zv_colnames, 
                  bw_columns = bw_fw_colnames, 
                  col_color_key = 'FW_velocitycmsecmean', 
                  units = units, 
                  output_folder_path = bland_alt_out_path, 
                  subfolder_name = 'zeno_fw_bland_alt_by_fw_vel')